# Fine-tuning GPT-3 for 💮 GenerAd 💮!

We'll walk through another example of fine-tuning, this time we'll use OpenAI's GPT-3 model. 

### Getting Requirements

In [ ]:
!pip install openai

In [ ]:
import openai
openai.api_key = ""

### Setting up Data

We'll want to ensure our data is in the following format as per OpenAI [suggestions](https://platform.openai.com/docs/guides/fine-tuning): 

```
{"prompt" : "<PRODUCT NAME>: <PRODUCT DESCRIPTION>\n\n###\n\n", "completion" : "<SWEET AD> END"}
```

So we'll need to transform our data into this format.

### Logging Into Hugging Face

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /home/chris/.cache/huggingface/token
Login successful


In [ ]:
from datasets import load_dataset

dataset = load_dataset("c-s-ale/Product-Descriptions-and-Ads")

Found cached dataset parquet (/home/chris/.cache/huggingface/datasets/c-s-ale___parquet/c-s-ale--Product-Descriptions-and-Ads-e60cdaa742e1bbad/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def transform_to_openai_format(row):
    return {"prompt" : f"{row['product']}: {row['description']}\n\n###\n\n", "completion" : f"{row['ad']} END"}

Now we can map the data to the format we need, and save it to a file in the format OpenAI expects.

In [ ]:
import json

openai_dataset = [transform_to_openai_format(row) for row in dataset["train"]]

with open("openai_dataset.jsonl", "w") as f:
    for row in openai_dataset:
        f.write(json.dumps(row) + "\n")

### Prepare to Fine-tune!

First, we'll ensure our data is in the correct format.

In [ ]:
!openai tools fine_tunes.prepare_data -f openai_dataset.jsonl -q

Analyzing...

- Your file contains 90 prompt-completion pairs. In general, we recommend having at least a few hundred examples. We've found that performance tends to linearly increase for every doubling of the number of examples
- All prompts end with suffix `.\n\n###\n\n`
- All prompts start with prefix ` `
- All completions end with suffix ` END`
- The completion should start with a whitespace character (` `). This tends to produce better results due to the tokenization we use. See https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more details

Based on the analysis we will perform the following actions:
- [Recommended] Add a whitespace character to the beginning of the completion [Y/n]: Y


Your data will be written to a new JSONL file. Proceed [Y/n]: Y

Wrote modified file to `openai_dataset_prepared (1).jsonl`
Feel free to take a look!

Now use that file when fine-tuning:
> openai api fine_tunes.create -t "openai_dataset_prepared (1).jsonl"

After you’

Now, we can load our OPENAI_API_KEY into our environment variables and fine-tune!

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
!openai api fine_tunes.create -t "openai_dataset_prepared.jsonl" -m "ada" --suffix "GenerAd"

You can continue to watch your model fine-tune with the following command:

In [ ]:
!openai api fine_tunes.follow -i <YOUR FINETUNE ID>

### Trying it out!

In [ ]:
model = "YOUR MODEL HERE"

#### Example in Training Set

In [ ]:
openai.Completion.create(
    model = model,
    prompt = "Lace-up sandals: Shoes featuring laces or ties that wrap around the foot and, in some cases, the ankle.\n\n###\n\n",
    max_tokens = 50,
)["choices"][0]["text"]

' Discover our new Lace-up Sandals - the perfect blend of style and comfort! Wrap your feet in comfortable shorts or tops that provide azuweki-style rope-style comfort. Shop now and turn heads - #ToughStockPower'

In [ ]:
openai.Completion.create(
    model = model,
    prompt = "Sundress: A flowery yellow sundress with blue polka dots.\n\n###\n\n",
    max_tokens = 50,
)["choices"][0]["text"]

" Stay warm & stylish this season with our adorable Breakaway Sundress. Party on top of the girl's cool, purple polka dots. Don't let the chill go, turn up on your way to sparkly success! Get yours now! END"

In [ ]:
openai.Completion.create(
    model = model,
    prompt = "A new Lexus: A luxury automobile with grey paint and tinted windows.\n\n###\n\n",
    max_tokens = 50,
)["choices"][0]["text"]

' Upgrade your lifestyle by adding style and sophistication to your Lexus brand as we launch a new model: Grey & Tinted! Experience comfort & sophistication with strong sobriety standards. Get yours today! #LexusYearmondiff #WithH'